In [218]:
import g
def read_squad_examples(input_file, debug=False):
    # Read data
    unproc_data = []
    with gzip.open(input_file, 'rt', encoding='utf-8') as f:  # opening file in binary(rb) mode
        for item in json_lines.reader(f):
            # print(item) #or use print(item['X']) for printing specific data
            unproc_data.append(item)

    # Delete header
    unproc_data = unproc_data[1:]
    if debug:
        unproc_data = unproc_data[:100]

    ###################### Make Examples ######################
    examples = []
    for item in unproc_data:
        # 1. Get Context
        doc_tokens = []
        for token in item['context_tokens']:
            # BERT has only [SEP] in it's word piece vocabulary. because we keps all separators char length 5
            # we can replace all of them with [SEP] without modifying the offset
            if token[0] in ['[TLE]', '[PAR]', '[DOC]']:
                token[0] = '[SEP]'
            doc_tokens.append(token[0])

        # 2. qas
        for qa in item['qas']:
            qas_id = qa['qid']
            question_text = qa['question']

            answer_lst = []  # Check for duplicate question
            for answer in qa['detected_answers']:
                orig_answer_text = answer['text']
                # We could find so many duplicate "Detected Answer"...It needs to be erased
                if orig_answer_text in answer_lst:
                    continue
                else:
                    answer_lst.append(orig_answer_text)

                # Only take the first span
                start_position = answer['token_spans'][0][0]
                end_position = answer['token_spans'][0][1]

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position)
                examples.append(example)
    return examples

In [220]:
read_squad_examples("datasets/race")

NameError: name 'gzip' is not defined